In [33]:
collection = {
    ('Document1', 'This is a a a a a a a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not a sample')
}

In [34]:
from collections import defaultdict
from math import log2
from konlpy.corpus import kobill
from konlpy.tag import Komoran

DTM = defaultdict(lambda:defaultdict(int))  
for docName, docContent in collection:
    for term in docContent.lower().split():
        DTM[docName][term] += 1

TDM = defaultdict(lambda:defaultdict(int))
for idx, termList in DTM.items():
    for term, freq in termList.items():
        TDM[term][idx] = freq
        
TWM = defaultdict(lambda:defaultdict(float))
N = len(DTM)
for idx, termList in DTM.items():
    maxTF = max(termList.values())
    for term, freq in termList.items():
        TF = freq/maxTF
        IDF = log2(N/len(TDM[term]))
        TWM[term][idx] = TF*IDF

In [35]:
# DTM은 각 문서마다 term을 저장
DTM 

defaultdict(<function __main__.<lambda>()>,
            {'Document3': defaultdict(int,
                         {'this': 1, 'is': 1, 'not': 1, 'a': 1, 'sample': 1}),
             'Document1': defaultdict(int,
                         {'this': 1, 'is': 1, 'a': 7, 'sample': 1}),
             'Document2': defaultdict(int,
                         {'this': 1, 'is': 1, 'another': 1, 'sample': 1})})

In [36]:
# TDM은 각 용어가 어느 document에 몇개씩 있는지 저장
TDM

defaultdict(<function __main__.<lambda>()>,
            {'this': defaultdict(int,
                         {'Document3': 1, 'Document1': 1, 'Document2': 1}),
             'is': defaultdict(int,
                         {'Document3': 1, 'Document1': 1, 'Document2': 1}),
             'not': defaultdict(int, {'Document3': 1}),
             'a': defaultdict(int, {'Document3': 1, 'Document1': 7}),
             'sample': defaultdict(int,
                         {'Document3': 1, 'Document1': 1, 'Document2': 1}),
             'another': defaultdict(int, {'Document2': 1})})

In [37]:
# 각 term의 tf-idf 계산하여 저장
# tf - 한 document 내에서 그 용어 나오는 횟수 나누기 총 용어 갯수
# idf - 특정 용어가 다른 document들에서 얼마나 나오는지 계산 => 다른 document에서 많이 나올수록
#       덜 중요한 용어로 생각 ex) the, a, he 등등
TWM

defaultdict(<function __main__.<lambda>()>,
            {'this': defaultdict(float,
                         {'Document3': 0.0,
                          'Document1': 0.0,
                          'Document2': 0.0}),
             'is': defaultdict(float,
                         {'Document3': 0.0,
                          'Document1': 0.0,
                          'Document2': 0.0}),
             'not': defaultdict(float, {'Document3': 1.584962500721156}),
             'a': defaultdict(float,
                         {'Document3': 0.5849625007211562,
                          'Document1': 0.5849625007211562}),
             'sample': defaultdict(float,
                         {'Document3': 0.0,
                          'Document1': 0.0,
                          'Document2': 0.0}),
             'another': defaultdict(float, {'Document2': 1.584962500721156})})

똑같이 kobill에도 적용해보자

In [57]:
from math import sqrt

ma = Komoran()


# .txt 파일마다 형태소 분석하여 저장
DTM = defaultdict(lambda:defaultdict(int))  
for idx in kobill.fileids():
#     ma.pos() => (형태소, 품사)
#     ma.morphs() => 형태소, 형태소, ...
#     ma.nouns() => 명사, 명사, ...
    for term in kobill.open(idx).read().split():
        for token in ma.pos(term):
            DTM[idx]['/'.join(token)] += 1
            # '형태소/품사'

TDM = defaultdict(lambda:defaultdict(int))
DVL = defaultdict(float)
for idx, termList in DTM.items():
    for term, freq in termList.items():
        TDM[term][idx] = freq
        
TWM = defaultdict(lambda:defaultdict(float))
N = len(DTM)
for idx, termList in DTM.items():
    maxTF = max(termList.values())
    for term, freq in termList.items():
        TF = freq/maxTF
        IDF = log2(N/len(TDM[term]))
        TWM[term][idx] = TF*IDF
        DVL[idx] += TWM[term][idx]**2
        
for idx, length in DVL.items():
    DVL[idx] = sqrt(length)  

In [78]:
from nltk.tokenize import sent_tokenize

query = '국방의 의무와 보편적 의무에 대한 의무를 찾아주세요.'

TQM = defaultdict(int)
QWM = defaultdict(float)

for term in query.split():
    for token in ma.pos(term):
        TQM['/'.join(token)] += 1
        
alpha = 0.5
maxTF = max(TQM.values())
for term, ferq in TQM.items():
    TF = alpha + (1-alpha)*(freq/maxTF)
    DF = len(TWM[term]) if len(TWM[term]) > 0 else 1
    IDF = log2(N/DF)
    QWM[term] = TF*IDF
    
candidateList = defaultdict(float)
for term, weight1 in QWM.items():
    for doc, weight2 in TWM[term].items():
        innerProduct = weight1 * weight2
        candidateList[doc] += innerProduct
        
for doc, sim in candidateList.items():
    candidateList[doc] = sim/DVL[doc]
    
K = 5
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1], reverse=True)[:K]:
    print('문서이름:{0} / 유사도:{1:.4f}'.format(doc, sim))
    print(sent_tokenize(kobill.open(doc).read())[:3])
    print()

문서이름:1809899.txt / 유사도:0.0509
['결혼중개업의 관리에 관한 법률 일부개정법률안\n\n(한선교의원 대표발의 )\n\n 의 안\n 번 호\n\n9899\n\n발의연월일 : 2010.', '11.', '15.']

문서이름:1809898.txt / 유사도:0.0215
['국군부대의 소말리아 해역 파견연장 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

문서이름:1809897.txt / 유사도:0.0204
['국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 \n관한 파견 동의안\n\n의안\n                                                                  제출연월일 :  2010.', '11.', '15.']

문서이름:1809891.txt / 유사도:0.0040
['국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809890.txt / 유사도:0.0040
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.']



In [80]:
candidateList = defaultdict(float)
for term, docList in TWM.items():
    for doc, weight1 in docList.items():
        weight2 = QWM[term]
        candidateList[doc] += (weight1 - weight2)**2
        
for doc, sim in candidateList.items():
    candidateList[doc] = sqrt(sim)

for doc in DTM:
    print(doc, len(kobill.open(doc).read().split()), len(DTM[doc]), sum(DTM[doc].values()))
candidateList

1809890.txt 841 484 1947
1809891.txt 834 480 1941
1809892.txt 984 516 2250
1809893.txt 840 490 1942
1809894.txt 242 206 520
1809895.txt 394 272 889
1809896.txt 1939 624 4217
1809897.txt 788 409 1430
1809898.txt 821 405 1466
1809899.txt 1677 532 3899


defaultdict(float,
            {'1809890.txt': 1.5776888903870359,
             '1809891.txt': 1.5764702195076241,
             '1809892.txt': 1.4514868834631391,
             '1809893.txt': 1.658477051331954,
             '1809897.txt': 3.34281895173356,
             '1809898.txt': 2.9672702012942676,
             '1809894.txt': 3.60118175906683,
             '1809895.txt': 2.6022726333121886,
             '1809896.txt': 2.0707650112128686,
             '1809899.txt': 3.0026304064289966})

In [83]:
from nltk.tokenize import sent_tokenize

K = 5
for doc, sim in sorted(candidateList.items(), key=lambda x:x[1])[:K]:
    print("문서이름:{0} / 거리:{1:.4f}".format(doc, sim))
    print(sent_tokenize(kobill.open(doc).read())[:3])
    print()

문서이름:1809892.txt / 거리:1.4515
['교육공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9892\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809891.txt / 거리:1.5765
['국가공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9891\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809890.txt / 거리:1.5777
['지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9890\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809893.txt / 거리:1.6585
['남녀고용평등과 일 ·가정 양립 지원에 관한 법률 \n\n일부개정법률안\n\n(정의화의원 대표발의 )\n\n 의 안\n 번 호\n\n9893\n\n발의연월일 : 2010.', '11.', '12.']

문서이름:1809896.txt / 거리:2.0708
['행정절차법 일부개정법률안\n\n(유선호의원 대표발의 )\n\n 의 안\n 번 호\n\n9896\n\n발의연월일 : 2010.', '11.', '15.']

